In [3]:
import os
import pandas as pd
import dask.dataframe as dd
from dask import delayed

def cargar_datos():
    data_path = '../data'
    files = os.listdir(data_path)
    
    # Filtra y recopila únicamente los archivos que cumplen con la convención de nombres
    selected_files = []
    for file in files:
        if file.endswith('.csv'):
            parts = file.split('_')
            if len(parts) < 3:
                continue
            year = parts[0]
            if year not in ['2024']:
                continue
            selected_files.append(os.path.join(data_path, file))
    
    # Función para procesar cada archivo, decorada con @delayed para que se ejecute de forma perezosa
    @delayed
    def process_file(file_path):
        try:
            df = pd.read_csv(file_path, low_memory=True, dtype=str)
            df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s', errors='coerce')
            df['year'] = df['last_reported'].dt.year
            df['month'] = df['last_reported'].dt.month
            df['day'] = df['last_reported'].dt.day
            df['hour'] = df['last_reported'].dt.hour

            # Elimina columnas innecesarias si existen
            cols_to_drop = [col for col in ['traffic', 'V1'] if col in df.columns]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

            # Convierte a float las columnas numéricas
            numeric_cols = ['num_bikes_available', 'num_docks_available', 
                            'num_bikes_available_types.mechanical', 'num_bikes_available_types.ebike']
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = df[col].astype(float)
            return df
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            return pd.DataFrame()  # Devuelve un DataFrame vacío en caso de error

    # Crea una lista de DataFrames diferidos (lazy)
    delayed_dfs = [process_file(file_path) for file_path in selected_files]
    
    if not delayed_dfs:
        return None

    # Convierte los objetos diferidos en un Dask DataFrame
    ddf = dd.from_delayed(delayed_dfs)

    # Carga la metadata de estaciones en un DataFrame de pandas (pequeño)
    df_meta = pd.read_csv('../Informacio_Estacions_Bicing_2025.csv',
                          usecols=['station_id', 'lat', 'lon', 'capacity'],
                          low_memory=False)
    # Asegúrate de que los tipos de 'station_id' coincidan
    ddf['station_id'] = ddf['station_id'].astype('Int64')
    df_meta['station_id'] = df_meta['station_id'].astype('Int64')

    # Procesa columnas que necesitan conversiones de tipo
    for col in ['is_installed', 'is_renting', 'is_returning']:
        if col in ddf.columns:
            ddf[col] = ddf[col].fillna(0).astype(int)
    
    if 'is_charging_station' in ddf.columns:
        ddf['is_charging_station'] = ddf['is_charging_station'].map({'TRUE': 1, 'FALSE': 0})
    
    # Realiza el merge con la metadata de estaciones (la metadata se carga en memoria)
    ddf = ddf.merge(df_meta, on='station_id', how='inner')
    
    # Calcula una columna adicional de capacidad total
    ddf['sum_capacity'] = ddf['num_bikes_available'] + ddf['num_docks_available']
    
    # Convierte a pandas para operaciones que se realizan de forma vectorizada
    df_final = ddf.compute()
    
    # Reemplaza los valores nulos en 'capacity' utilizando la mediana por estación (operación vectorizada)
    median_capacity = df_final.groupby('station_id')['sum_capacity'].median()
    df_final['capacity'] = df_final['capacity'].fillna(df_final['station_id'].map(median_capacity))
    
    # Limita 'num_docks_available' entre 0 y 'capacity' usando la función clip, que es vectorizada
    df_final['num_docks_available'] = df_final['num_docks_available'].clip(lower=0, upper=df_final['capacity'])
    
    # Calcula la variable objetivo
    df_final['target'] = df_final['num_docks_available'] / df_final['capacity']
    
    # Agrega los datos por estación y componentes temporales
    aggregated_df = df_final.groupby(['station_id', 'year', 'month', 'day', 'hour']).agg(
        num_bikes_available=('num_bikes_available', 'mean'),
        num_docks_available=('num_docks_available', 'mean'),
        num_mechanical=('num_bikes_available_types.mechanical', 'median'),
        num_ebike=('num_bikes_available_types.ebike', 'median'),
        is_renting=('is_renting', 'mean'),
        is_returning=('is_returning', 'mean'),
        target=('target', 'mean'),
        lat=('lat', 'first'),
        lon=('lon', 'first'),
        capacity=('capacity', 'first')
    ).reset_index()

    # Filtra por las estaciones disponibles en la metadata de envío
    id_df = pd.read_csv('../data/metadata_sample_submission_2025.csv')
    station_list = pd.unique(id_df['station_id'])
    aggregated_df = aggregated_df[aggregated_df['station_id'].isin(station_list)]
    
    return aggregated_df



In [4]:
#AFEGIM ALTRES VARIABLES ( 4h anteriors + tipo de dia: festiu cap setmana laborable)

def crear_campos_lags(df):
   # df= df.iloc[4:] 
  
    resultados = []
    # Agrupamos por station_id
    for station, grupo in df.groupby('station_id'):
        # Aseguramos el orden cronológico
        grupo = grupo.sort_values(by=['year', 'month', 'day', 'hour']).reset_index(drop=True)
        n = len(grupo)
        # Iteramos empezando en el índice 4 y avanzamos de 5 en 5
        for i in range(4, n):
            if i - 4 >= 0:
                fila = grupo.loc[i].copy()
                # Agregamos los valores de 'target' de las 4 horas previas
                fila['ctx-4'] = grupo.loc[i - 4, 'target']
                fila['ctx-3'] = grupo.loc[i - 3, 'target']
                fila['ctx-2'] = grupo.loc[i - 2, 'target']
                fila['ctx-1'] = grupo.loc[i - 1, 'target']
                
                resultados.append(fila)
    return pd.DataFrame(resultados)


#funcio per afegir el tipo de dia, festiu laborable o cap setmana 
def day_categorization_bcn(df):
    """
    Añade una columna 'day_type' al DataFrame con la clasificación numérica de cada día:
    0 = Laborable, 1 = Fin de semana, 2 = Festivo.
    """
    import pandas as pd

    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

    # Festivos en Barcelona (2020 - marzo 2025)
    holidays = [
        "2020-01-01", "2020-01-06", "2020-04-10", "2020-04-13", "2020-05-01", "2020-06-24", "2020-09-11", "2020-09-24",
        "2020-10-12", "2020-11-01", "2020-12-06", "2020-12-08", "2020-12-25", "2020-12-26",
        "2021-01-01", "2021-01-06", "2021-04-02", "2021-04-05", "2021-05-01", "2021-06-24", "2021-09-11", "2021-09-24",
        "2021-10-12", "2021-11-01", "2021-12-06", "2021-12-08", "2021-12-25", "2021-12-26",
        "2022-01-01", "2022-01-06", "2022-04-15", "2022-04-18", "2022-05-01", "2022-06-24", "2022-09-11", "2022-09-24",
        "2022-10-12", "2022-11-01", "2022-12-06", "2022-12-08", "2022-12-25", "2022-12-26",
        "2023-01-01", "2023-01-06", "2023-04-07", "2023-04-10", "2023-05-01", "2023-06-24", "2023-09-11", "2023-09-24",
        "2023-10-12", "2023-11-01", "2023-12-06", "2023-12-08", "2023-12-25", "2023-12-26",
        "2024-01-01", "2024-01-06", "2024-03-29", "2024-04-01", "2024-05-01", "2024-06-24", "2024-09-11", "2024-09-24",
        "2024-10-12", "2024-11-01", "2024-12-06", "2024-12-08", "2024-12-25", "2024-12-26",
        "2025-01-01", "2025-01-06", "2025-04-18", "2025-04-21", "2025-05-01", "2025-06-24", "2025-09-11", "2025-09-24",
        "2025-10-12", "2025-11-01", "2025-12-06", "2025-12-08", "2025-12-25", "2025-12-26"
    ]
    
    holiday_dates = pd.to_datetime(holidays)

    # Función para clasificar el día en valores numéricos
    def classify_day(date):
        if date in holiday_dates:
            return 2  # Festivo
        elif date.weekday() >= 5:  # Sábado (5) o domingo (6)
            return 1  # Fin de semana
        else:
            return 0  # Laborable

    df['day_type'] = df['date'].apply(classify_day)

    df.drop(columns=['date'], inplace=True)

    return df



In [6]:
df_merge = cargar_datos()

C:\Users\Incapto\AppData\Local\Temp\ipykernel_2116\1902172043.py:27: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s', errors='coerce')
c:\Users\Incapto\anaconda3\Lib\site-packages\dask_expr\_collection.py:4160: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('is_charging_station', 'float64'))

  warnings.warn(meta_warning(meta))
C:\Users\Inca

In [ ]:
df_merge= crear_campos_lags(df_merge)

In [ ]:
df_merge_final=day_categorization_bcn(df_merge)

C:\Users\Incapto\AppData\Local\Temp\ipykernel_14140\283822778.py:31: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s', errors='coerce')
c:\Users\Incapto\anaconda3\Lib\site-packages\dask_expr\_collection.py:4160: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('is_charging_station', 'float64'))

  warnings.warn(meta_warning(meta))
C:\Users\Inca

KeyboardInterrupt: 

In [8]:
# Ahora con el df_merge_final crear un df nuevo que sea df_train que coja 1 de cada 5 filas
df_train = df_merge_final.iloc[::5]
df_train = df_train.reset_index(drop=True)
print(df_train.head())  # Ver las primeras filas del nuevo DataFrame

   station_id    year  month  day  hour  num_bikes_available  \
0         1.0  2024.0    1.0  1.0   2.0            14.500000   
1         1.0  2024.0    1.0  1.0   7.0            13.000000   
2         1.0  2024.0    1.0  1.0  12.0             4.666667   
3         1.0  2024.0    1.0  1.0  17.0             5.666667   
4         1.0  2024.0    1.0  1.0  22.0            10.000000   

   num_docks_available  num_mechanical  num_ebike  is_renting  is_returning  \
0            30.500000             5.5        9.5         1.0           1.0   
1            32.000000             9.0        4.0         1.0           1.0   
2            40.333333             1.0        4.0         1.0           1.0   
3            39.333333             2.0        4.0         1.0           1.0   
4            35.000000             4.0        6.0         1.0           1.0   

     target        lat       lon  capacity     ctx-4     ctx-3     ctx-2  \
0  0.663043  41.397978  2.180107      46.0  0.717391  0.768116  

In [9]:
# Para separar los datos usamos df_train
X = df_train[['station_id','month', 'day', 'hour', 'ctx-1', 'ctx-2', 'ctx-3', 'ctx-4','lat','lon', 'day_type']]
y = df_train['target']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# Linear Regression

def linear_regression(X,y, size):
#Separamos train y test
    X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size= size, random_state=42)

    lm = LinearRegression()
#Entrenamiento modelo
    lm.fit(X_train, y_train)
#Predict
    y_pred = lm.predict(X_test)
#Metricas
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    return  r2, mse, mae


# Random Forest

def RandomForest(X,y, size):
#Separamos train y test
    X_train, X_test, y_train, y_test = train_test_split(X ,y , test_size= size, random_state=42)

    rf = RandomForestRegressor(n_estimators = 100, random_state= 42)
#Entrenamiento modelo
    rf.fit(X_train, y_train)
#Predict
    y_pred = rf.predict(X_test)
#Metricas
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
#Factoring Importance
    feature_importance = rf.feature_importances_
    feature_names = X_test.columns
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    return  r2, mse, mae, importance_df, rf

# Neural Network

def neural_network_model(X, y, size):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=42)
    
    # Define the neural network model; here we use a single hidden layer with 100 neurons.
    nn = MLPRegressor(hidden_layer_sizes=(100,),
                      activation='relu',
                      solver='adam',
                      random_state=42,
                      max_iter=500)
    
    # Train the model
    nn.fit(X_train, y_train)
    
    # Make predictions
    y_pred = nn.predict(X_test)
    
    # Calculate evaluation metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    return r2, mse, mae, nn


In [ ]:
r2_linear, mse_linear, mae_linear = linear_regression(X,y, 0.3)
r2_rf, mse_rf, mae_rf, importance_df,rf = RandomForest(X, y, 0.3)
r2_nn, mse_nn, mae_nn, nn_model = neural_network_model(X, y, 0.3)

metrics_df = pd.DataFrame({
    'Métrica': ['R²', 'MSE', 'MAE'],
    'Regresión Lineal': [r2_linear, mse_linear, mae_linear],
    'Random Forest': [r2_rf, mse_rf, mae_rf]
    'Neural Network' = [r2_nn, mse_nn, mae_nn]
})

print(metrics_df)
print("Importancia de las características según Random Forest:")
print(importance_df)

  Métrica  Regresión Lineal  Random Forest
0      R²          0.813425       0.821507
1     MSE          0.011177       0.010693
2     MAE          0.071522       0.070555
Importancia de las características:
       Feature  Importance
4        ctx-1    0.826631
5        ctx-2    0.038013
6        ctx-3    0.022586
7        ctx-4    0.021565
0   station_id    0.019756
9          lon    0.019517
8          lat    0.017714
3         hour    0.017167
2          day    0.014763
10    day_type    0.002288
1        month    0.000000


In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('../metadata_sample_submission_2025.csv')
df['year'] = 2025 #em faltava crear columna de year=2025 per poder aplicar funcio day_categorization_bcn 


# Función para imputar las columnas 'lat, lon' del df_merge_final en base (FK) la columa station_id
# ---- IMPUTACIÓN DE COLUMNAS ADICIONALES ----
cols_to_impute = ['lat', 'lon']

# Usamos `merge` para traer la información de df_merge_final a df
df = df.merge(df_merge_final[['station_id'] + cols_to_impute], on='station_id', how='left')
df= day_categorization_bcn(df) 

# Seleccionar las características para predecir
X_predict = df[['station_id', 'month', 'day', 'hour', 'ctx-4', 'ctx-3', 'ctx-2', 'ctx-1','lat','lon', 'day_type']]

# Hacer predicciones
predictions = rf.predict(X_predict)

# Añadir las predicciones como una nueva columna en el DataFrame
df['percentage_docks_available'] = predictions

# Guardar el DataFrame con las predicciones en un nuevo archivo CSV
df.to_csv('metadata_sample_submission_with_predictions.csv', index=False)

print("Predicciones añadidas y guardadas en 'metadata_sample_submission_with_predictions.csv'")

: 

In [ ]:
df_final = df[['index','percentage_docks_available']]
df_final.to_csv('predictions.csv', index=False)